In [ ]:
import os
cwd=os.getcwd()
os.chdir(os.path.dirname(cwd))
print(os.getcwd())
import json
from core.data_ingestion import crunchbase
import dotenv
dotenv.load_dotenv()


%load_ext autoreload
%autoreload 2

In [7]:
import psycopg2
conn = psycopg2.connect(
    dbname="startup_database",
    user="postgres",
    password=os.getenv("DB_PASSWORD"),
    host="localhost",
    port="5432"
)

In [8]:
cur=conn.cursor()

In [11]:
cur.execute("SELECT * FROM industry_connections")
industry_connections=cur.fetchall()

In [8]:
# delete all rows from industry_connections
cur.execute("DELETE FROM industry_connections")
conn.commit()

In [37]:
# join 
cur.execute("""
SELECT crunchbase.company_name, industry_connections.industry_name
FROM crunchbase
JOIN industry_connections ON crunchbase.primary_key = industry_connections.crunchbase_primary_key"""
)
rows=cur.fetchall()


In [ ]:
industry_set=set()
for row in rows:
    industry_set.add(row[1])
print(len(industry_set))

In [ ]:
cur.execute("""SELECT industry_connections.industry_name, COUNT(ci.company_id) as company_count
FROM company_industries ci
JOIN industries i ON ci.industry_id = i.id
GROUP BY i.industry_name
ORDER BY company_count DESC
LIMIT 10;""")
data=cur.fetchall()

In [48]:
cur.execute("""SELECT industry_name, COUNT(*) as company_count
FROM industry_connections
GROUP BY industry_name
ORDER BY company_count DESC
LIMIT 15;""")
data=cur.fetchall()

In [24]:
# same as above but only the top 100 when sorted by cb_rank in acending order
cur.execute("""SELECT industry_name, COUNT(*) as company_count
FROM (
    SELECT ic.industry_name, c.cb_rank
    FROM industry_connections ic
    JOIN crunchbase c ON ic.crunchbase_primary_key = c.primary_key
    ORDER BY c.cb_rank
    LIMIT 200
) top_companies
GROUP BY industry_name
ORDER BY company_count DESC
LIMIT 100;""")


In [31]:
cur.execute("""WITH top_companies AS (
    SELECT primary_key
    FROM crunchbase
    ORDER BY cb_rank
    LIMIT 100
)
SELECT ic.industry_name, COUNT(*) as company_count
FROM industry_connections ic
JOIN top_companies tc ON ic.crunchbase_primary_key = tc.primary_key
GROUP BY ic.industry_name
ORDER BY company_count DESC
LIMIT 100;""")
keys=[]
values=[]
for industry, count in data:
    keys.append(industry)
    values.append(count)
    

In [28]:
data=cur.fetchall()

In [ ]:
data[][0]

In [ ]:
print(len(data))
data